<a href="https://colab.research.google.com/github/osmarbraz/exemplos_BERT/blob/main/ExemplosWordEmbeddingContextualBERT_pt_br_sentenca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Word Embeddings(pt-br) Contextual usando BERT Transformers by HuggingFace

## **A execução pode ser feita através do menu Ambiente de Execução opção Executar tudo.**

Exemplos de uso de **Word Embeddings Contextuais** para **desambiguação** de documentos originais e permutados utilizando suas sentenças. No final do notebook estão os exemplos com os documentos:

*   documento original e permutado

**Link biblioteca Huggingface:**
https://github.com/huggingface/transformers


**Artigo original BERT Jacob Devlin:**
https://arxiv.org/pdf/1506.06724.pdf

## 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

###Tratamento de logs

Método para tratamento dos logs.

In [1]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [2]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = 'google.colab' in sys.modules

## 1 - Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [3]:
# Instala a última versão da biblioteca
#!pip install transformers

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.5.1

Requirement already up-to-date: transformers==4.5.1 in /usr/local/lib/python3.7/dist-packages (4.5.1)


## 2 - Download do arquivo do PyTorch Checkpoint

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **'neuralmind/bert-base-portuguese-cased'**
* **'neuralmind/bert-large-portuguese-cased'**

In [4]:
# Importando as bibliotecas
import os

# Variável para setar o arquivo
URL_MODELO = None

# Comente uma das urls para carregar modelos de tamanhos diferentes(base/large)
# URL_MODELO do arquivo do modelo tensorflow
# arquivo menor(base) 1.1 Gbytes
# URL_MODELO = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip"

# arquivo grande(large) 3.5 Gbytes
URL_MODELO = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip"

# Se a variável foi setada
if URL_MODELO:

    # Diretório descompactação
    DIRETORIO_MODELO = '/content/modelo'

    # Recupera o nome do arquivo do modelo da URL_MODELO
    arquivo = URL_MODELO.split("/")[-1]

    # Nome do arquivo do vocabulário
    arquivo_vocab = "vocab.txt"

    # Caminho do arquivo na URL_MODELO
    caminho = URL_MODELO[0:len(URL_MODELO)-len(arquivo)]

    # Verifica se a pasta de descompactação existe no pasta corrente
    if not os.path.exists(DIRETORIO_MODELO):
   
        # Baixa o arquivo do modelo
        !wget $URL_MODELO
    
        # Descompacta o arquivo na pasta de descompactação
        !unzip -o $arquivo -d $DIRETORIO_MODELO

        # Baixa o arquivo do vocabulário
        # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente
        URL_MODELO_VOCAB = caminho + arquivo_vocab
        !wget $URL_MODELO_VOCAB
    
        # Coloca o arquivo do vocabulário no diretório de descompactação
        !mv $arquivo_vocab $DIRETORIO_MODELO
            
        # Move o arquivo para pasta de descompactação
        !mv $arquivo $DIRETORIO_MODELO
       
        print('Pasta do ' + DIRETORIO_MODELO + ' pronta!')
    else:
      print('Pasta do ' + DIRETORIO_MODELO + ' já existe!')

    # Lista a pasta corrente
    !ls -la $DIRETORIO_MODELO
else:
    print('Variável URL_MODELO não setada!')

Pasta do /content/modelo já existe!
total 2525908
drwxr-xr-x 2 root root       4096 May 20 17:23 .
drwxr-xr-x 1 root root       4096 May 20 17:23 ..
-rw-r--r-- 1 root root 1244275810 Jan 22  2020 bert-large-portuguese-cased_pytorch_checkpoint.zip
-rw-rw-r-- 1 root root        874 Jan 12  2020 config.json
-rw-rw-r-- 1 root root 1342014951 Jan 12  2020 pytorch_model.bin
-rw-r--r-- 1 root root     209528 Jan 21  2020 vocab.txt


## 3 - Carregando o Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).

Carregando o tokenizador da pasta '/content/modelo/' do diretório padrão se variável `URL_MODELO` setada.

**Caso contrário carrega da comunidade**

Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.

O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado apartir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.

In [5]:
# Importando as bibliotecas do tokenizador
from transformers import BertTokenizer

# Se a variável URL_MODELO foi setada
if URL_MODELO:
    # Carregando o Tokenizador
    print('Carrgando o tokenizador BERT do diretório ' + DIRETORIO_MODELO + '...')

    tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, 
                                              do_lower_case=False)    
else:
    # Carregando o Tokenizador da comunidade
    print('Carregando o tokenizador da comunidade...')
    
    #tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
    tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', 
                                              do_lower_case=False)

Carrgando o tokenizador BERT do diretório /content/modelo...


## 4 - Carregando o Modelo BERT(BertModel)

Se a variável `URL_MODELO` estiver setada carrega o modelo do diretório `content/modelo`.

Caso contrário carrega da comunidade.

Carregando o modelo da pasta '/content/modelo/' do diretório padrão.

A implementação do huggingface pytorch inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PNL. Embora essas interfaces sejam todas construídas sobre um modelo treinado de BERT, cada uma possui diferentes camadas superiores e tipos de saída projetados para acomodar suas tarefas específicas de PNL.

A documentação para estas pode ser encontrada em [aqui](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html).

Por default o modelo está em modo avaliação ou seja `model.eval()`.

-----------------------

Durante a avaliação do modelo, este retorna um número de diferentes objetos com base em como é configurado na chamada do método `from_pretrained`. 

Quando definimos `output_hidden_states = True` na chamada do método `from_pretrained`, retorno do modelo possui no terceiro item os estados ocultos(**hidden_states**) de todas as camadas.  Veja a documentação para mais detalhes: https://huggingface.co/transformers/model_doc/bert.html#bertmodel

Quando **`output_hidden_states = True`** model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output; 
- outputs[2] = hidden_states.

Quando **`output_hidden_states = False`** ou não especificado model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output.


**ATENÇÃO**: O parâmetro ´**output_hidden_states = True**´ habilita gerar as camadas ocultas do modelo. Caso contrário somente a última camada é mantida. Este parâmetro otimiza a memória mas não os resultados.


In [6]:
# Importando as bibliotecas do Modelo
from transformers import BertModel

# Se a variável URL_MODELO1 foi setada
if URL_MODELO:
    # Carregando o Tokenizador
    print('Carregando o modelo BERT do diretório ' + DIRETORIO_MODELO + '...')

    model = BertModel.from_pretrained(DIRETORIO_MODELO, 
                                      output_attentions = True,
                                      output_hidden_states = True)    
else:
    # Carregando o Tokenizador da comunidade
    print('Carregando o modelo BERT da comunidade ...')

    model = BertModel.from_pretrained('neuralmind/bert-large-portuguese-cased', 
                                      output_attentions = True,
                                      output_hidden_states = True)

Carregando o modelo BERT do diretório /content/modelo...


## 5 - Funções auxiliares

### getEmbeddingsCamadas

Funções que recuperam os embeddings das camadas:
- Primeira camada;
- Penúltima camada;
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [7]:
def getEmbeddingPrimeiraCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][0]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado

def getEmbeddingPenultimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][-2]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado

def getEmbeddingUltimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
     
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][-1]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado    

def getEmbeddingSoma4UltimasCamadas(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  embeddingCamadas = output[2][-4:]
  # Saída: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  

  # Usa o método `stack` para criar uma nova dimensão no tensor 
  # com a concateção dos tensores dos embeddings.        
  #Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultadoStack = torch.stack(embeddingCamadas, dim=0)
  # Saída: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  
  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultadoStack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>
  
  return resultado

def getEmbeddingConcat4UltimasCamadas(output):  
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Cria uma lista com os tensores a serem concatenados
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  # Lista com os tensores a serem concatenados
  listaConcat = []
  # Percorre os 4 últimos
  for i in [-1,-2,-3,-4]:
      # Concatena da lista
      listaConcat.append(output[2][i])
  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  # Realiza a concatenação dos embeddings de todos as camadas
  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = torch.cat(listaConcat, dim=-1)
  # Saída: Entrada: (<1(lote)> x <qtde_tokens> <3072 ou 4096>)  
    
  return resultado   

def getEmbeddingSomaTodasAsCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
   
  # Retorna todas as camadas descontando a primeira(0)
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  embeddingCamadas = output[2][1:]
  # Saída: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  # Usa o método `stack` para criar uma nova dimensão no tensor 
  # com a concateção dos tensores dos embeddings.        
  #Entrada: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultadoStack = torch.stack(embeddingCamadas, dim=0)
  # Saída: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
    
  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultadoStack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>
    
  return resultado

### Imports

In [8]:
# Import das bibliotecas
import numpy as np
import torch

import matplotlib.pyplot as plt
%matplotlib inline

### getEmbeddingsVisual

Função para gerar as coordenadas de plotagem a partir das sentenças de embeddings.

Existe uma função para os tipos de camadas utilizadas:
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [9]:
def getEmbeddingsVisualUltimaCamada(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingUltimaCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

In [10]:
def getEmbeddingsVisualSoma4UltimasCamadas(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingSoma4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

In [11]:
def getEmbeddingsVisualConcat4UltimasCamadas(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingConcat4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

In [12]:
def getEmbeddingsVisualSomaTodasAsCamadas(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingSomaTodasAsCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

## 6 - Exemplo sentenças de documento original e permutado utilizando embedding da última camada do BERT

### Funções auxiliares

In [13]:
def getDocumentoTokenizado(documento, tokenizador):

    # Adiciona os tokens especiais.
    documentoMarcado = "[CLS] " + documento + " [SEP]"

    # Documento tokenizado
    documentoTokenizado = tokenizador.tokenize(documentoMarcado)

    return documentoTokenizado

In [14]:
# Localiza os índices de início e fim de uma sublista em uma lista
def encontrarIndiceSubLista(lista, sublista):
    # Recupera o tamanho da lista 
    h = len(lista)
    # Recupera o tamanho da sublista
    n = len(sublista)    
    skip = {sublista[i]: n - i - 1 for i in range(n - 1)}
    i = n - 1
    while i < h:
        for j in range(n):
            if lista[i - j] != sublista[-j - 1]:
                i += skip.get(lista[i], n)
                break
        else:
            indiceInicio = i - n + 1
            indiceFim = indiceInicio + len(sublista)-1
            return indiceInicio, indiceFim
    return -1, -1

In [15]:
def getEmbeddingSentencaDocumentoEmbedding(embeddingDocumento, documento, sentenca, tokenizador):
  # Tokeniza o documento
  documentoTokenizado = getDocumentoTokenizado(documento, tokenizador)  
  #print(documentoTokenizado)

  # Tokeniza a sentença
  sentençaTokenizada = getDocumentoTokenizado(sentenca, tokenizador)
  
  # Remove os tokens de início e fim da sentença
  sentençaTokenizada.remove('[CLS]')
  sentençaTokenizada.remove('[SEP]')  
  #print(sentençaTokenizada)
  #print(len(sentençaTokenizada))
  
  # Localiza os índices dos tokens da sentença no documento
  inicio, fim = encontrarIndiceSubLista(documentoTokenizado,sentençaTokenizada)
  #print(inicio,fim) 
 
  # Recupera os embeddings dos tokens da sentença a partir dos embeddings do documento
  embeddingSentenca = embeddingDocumento[inicio:fim+1]
  #print("embeddingSentenca=", embeddingSentenca.shape)
  
  # Retorna o embedding da sentença no documento
  return embeddingSentenca

### Documento Original

In [16]:
# Define um sentença de exemplo com diversos significados da palavra  "pilha"
documento_original = ["Bom Dia, professor.",
             "Qual o conteúdo da prova?",              
             "Vai cair tudo na prova?",
             "Aguardo uma resposta, João."]

# Concatena as sentenças do documento
documento_texto_original = ' '.join(documento_original)

# Adiciona os tokens especiais
documento_marcado_original = "[CLS] " + documento_texto_original + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_original = tokenizer.tokenize(documento_marcado_original)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_original = tokenizer.convert_tokens_to_ids(documento_tokenizado_original)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_original, documento_tokens_indexados_original):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Bom           8,399
  2 Dia           3,616
  3 ,               117
  4 professor     2,917
  5 .               119
  6 Qual         13,082
  7 o               146
  8 conteúdo      5,015
  9 da              180
 10 prova         2,310
 11 ?               136
 12 Vai          20,805
 13 cair          9,322
 14 tudo          2,745
 15 na              229
 16 prova         2,310
 17 ?               136
 18 Agu           8,125
 19 ##ardo        2,222
 20 uma             230
 21 resposta      4,299
 22 ,               117
 23 João          1,453
 24 .               119
 25 [SEP]           102


Máscara de atenção das palavras

In [17]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_original = [1] * len(documento_tokenizado_original)

print (mascara_atencao_original)
print (len(mascara_atencao_original))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [18]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_original = torch.as_tensor([documento_tokens_indexados_original])
mascara_atencao_tensores_original = torch.as_tensor([mascara_atencao_original])

Gera os embeddings para o documento original. Guarda somente a última camada da rede em `outputs`.


In [19]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_original, mascara_atencao_tensores_original)

Recupera a saída da última camada

In [20]:
# Recupera a última e única camada da saída
last_hidden_states = outputs[0]

print ("O vetor da última camada oculta tem o formato:", last_hidden_states.size())

O vetor da última camada oculta tem o formato: torch.Size([1, 26, 1024])


Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [21]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
token_embeddings_original = torch.squeeze(last_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento original tem o formato:", token_embeddings_original.size())

O vetor de tokens de embedding do documento original tem o formato: torch.Size([26, 1024])


Confirmando vetores dependentes do contexto


In [22]:
for i, token_str in enumerate(documento_tokenizado_original):
  print (i, token_str)

0 [CLS]
1 Bom
2 Dia
3 ,
4 professor
5 .
6 Qual
7 o
8 conteúdo
9 da
10 prova
11 ?
12 Vai
13 cair
14 tudo
15 na
16 prova
17 ?
18 Agu
19 ##ardo
20 uma
21 resposta
22 ,
23 João
24 .
25 [SEP]


Exibe os embenddings das sentenças

In [23]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca1Original, tokenizer)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca2Original, tokenizer)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca3Original, tokenizer)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca4Original, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento original.')

print('\nSentença 1:', sentenca1Original,'-', str(embeddingSentenca1Original[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Original,'-', str(embeddingSentenca2Original[:4]))
print('Soma embedding Sentença2:', sentenca2Original,'-', str(torch.sum(embeddingSentenca2Original[:4])))

print('\nSentença 3:', sentenca3Original,'-', str(embeddingSentenca3Original[:4]))
print('Soma embedding Sentença3:', sentenca3Original,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Original,'-', str(embeddingSentenca4Original[:4]))
print('Soma embedding Sentença4:', sentenca4Original,'-', str(torch.sum(embeddingSentenca4Original[:4])))

Os primeiros 4 valores de cada sentença do documento original.

Sentença 1: Bom Dia, professor. - tensor([[-0.5098, -0.2877,  0.0873,  ...,  0.5436, -0.9302,  0.4668],
        [ 0.6078, -0.8869,  0.3736,  ..., -0.3517, -1.2140, -0.3077],
        [ 0.9075, -1.1233, -0.0093,  ...,  0.4433, -0.4633, -0.0113],
        [ 0.2499, -0.4717, -0.1217,  ...,  0.7079, -0.2300,  0.4911]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-113.4554)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.3987, -0.9450,  0.1785,  ...,  0.7189, -0.6772, -0.1452],
        [ 0.2067, -0.2705,  0.7145,  ...,  0.3047, -0.2718,  0.7577],
        [ 0.2355,  0.2686,  0.5669,  ...,  1.0817,  0.5614,  0.3750],
        [ 0.5264, -0.4600,  0.4810,  ..., -0.5559, -0.2941,  0.0378]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-115.8800)

Sentença 3: Vai cair tudo na prova? - tensor([[ 0.5178,  0.0863,  0.7394,  ..., -0.5765, -0.6208, -0.2230],
        [ 0.1617,  1.1516, -0.0350,  ...,  0.1730,  

Examinando os embeddings do documento original



In [24]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

print("Documento Original:", documento_original)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaOriginal = tokenizer.tokenize(sentenca1Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca1TokenizadaOriginal)
embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca1Original, tokenizer)
print('\nSentença 1 Original=\'', sentenca1Original, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Original))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaOriginal = tokenizer.tokenize(sentenca2Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca2TokenizadaOriginal)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca2Original, tokenizer)
print('\nSentença 2 Original=\'', sentenca2Original, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Original))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaOriginal = tokenizer.tokenize(sentenca3Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca3TokenizadaOriginal)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca3Original, tokenizer)
print('\nSentença 3 Original=\'', sentenca3Original, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Original))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaOriginal = tokenizer.tokenize(sentenca4Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca4TokenizadaOriginal)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca4Original, tokenizer)
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))


Documento Original: ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']

Sentença 1 Original=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 1 e término em 5
    Formato modelo : torch.Size([5, 1024])
    Soma embeddings:  -141.45

Sentença 2 Original=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 6 e término em 11
    Formato modelo : torch.Size([6, 1024])
    Soma embeddings:  -173.58

Sentença 3 Original=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 12 e término em 17
    Formato modelo : torch.Size([6, 1024])
    Soma embeddings:  -167.58

Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 18 e término em 24
    Formato modelo : torch.S

### Documento Permutado

In [25]:
# Define um sentença de exemplo com diversos significados da palavra  "pilha"
documento_permutado = ["Aguardo uma resposta, João.",
             "Qual o conteúdo da prova?",              
             "Vai cair tudo na prova?",
             "Bom Dia, professor."]

# Use o documento permutado igual ao original para testar se as medidas estão corretas
#documento_permutado = documento_original

# Concatena as sentenças do documento
documento_texto_permutado = ' '.join(documento_permutado)

# Adiciona os tokens especiais
documento_marcado_permutado = "[CLS] " + documento_texto_permutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_permutado = tokenizer.tokenize(documento_marcado_permutado)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_permutado = tokenizer.convert_tokens_to_ids(documento_tokenizado_permutado)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_permutado, documento_tokens_indexados_permutado):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Agu           8,125
  2 ##ardo        2,222
  3 uma             230
  4 resposta      4,299
  5 ,               117
  6 João          1,453
  7 .               119
  8 Qual         13,082
  9 o               146
 10 conteúdo      5,015
 11 da              180
 12 prova         2,310
 13 ?               136
 14 Vai          20,805
 15 cair          9,322
 16 tudo          2,745
 17 na              229
 18 prova         2,310
 19 ?               136
 20 Bom           8,399
 21 Dia           3,616
 22 ,               117
 23 professor     2,917
 24 .               119
 25 [SEP]           102


In [26]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_permutado = [1] * len(documento_tokenizado_permutado)

print (mascara_atencao_permutado)
print (len(mascara_atencao_permutado))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [27]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_permutado = torch.as_tensor([documento_tokens_indexados_permutado])
mascara_atencao_tensores_permutado = torch.as_tensor([mascara_atencao_permutado])

Gera os embeddings para o documento original. Guarda somente a última camada da rede em `outputs`.


In [28]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_permutado, mascara_atencao_tensores_permutado)

Recupera a saída da última camada

In [29]:
# Recupera a última e única camada da saída
last_hidden_states = outputs[0]

print ("O vetor da última camada oculta tem o formato:", last_hidden_states.size())

O vetor da última camada oculta tem o formato: torch.Size([1, 26, 1024])


Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [30]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
token_embeddings_permutado = torch.squeeze(last_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento permutado tem o formato:", token_embeddings_permutado.size())

O vetor de tokens de embedding do documento permutado tem o formato: torch.Size([26, 1024])


Exibe os embenddings das sentenças

In [31]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca1Permutado, tokenizer)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca2Permutado, tokenizer)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca3Permutado, tokenizer)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca4Permutado, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento permutado.')

print('\nSentença 1:', sentenca1Permutado,'-', str(embeddingSentenca1Permutado[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Permutado,'-', str(embeddingSentenca2Permutado[:4]))
print('Soma embedding Sentença2:', sentenca2Permutado,'-', str(torch.sum(embeddingSentenca2Permutado[:4])))

print('\nSentença 3:', sentenca3Permutado,'-', str(embeddingSentenca3Permutado[:4]))
print('Soma embedding Sentença3:', sentenca3Permutado,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Permutado,'-', str(embeddingSentenca4Permutado[:4]))
print('Soma embedding Sentença4:', sentenca4Permutado,'-', str(torch.sum(embeddingSentenca4Permutado[:4])))

Os primeiros 4 valores de cada sentença do documento permutado.

Sentença 1: Aguardo uma resposta, João. - tensor([[ 0.4472, -0.7700, -0.0164,  ...,  0.3660, -1.3074,  0.4681],
        [-0.6242, -1.0259, -0.2210,  ..., -0.0656, -0.7453,  0.2866],
        [ 1.3008, -1.0530,  0.9163,  ...,  0.3736, -0.1157,  0.4558],
        [ 0.9188, -1.0255,  0.2020,  ...,  0.4574, -1.0556,  0.6117]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-113.4554)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.3900, -0.9698,  0.1223,  ...,  0.6340, -0.5703, -0.1435],
        [ 0.1376, -0.3465,  0.7442,  ...,  0.2438, -0.2685,  0.7738],
        [ 0.2753,  0.3924,  0.4863,  ...,  1.1585,  0.5529,  0.3655],
        [ 0.5488, -0.5081,  0.4872,  ..., -0.5818, -0.3959,  0.0555]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-116.1011)

Sentença 3: Vai cair tudo na prova? - tensor([[ 0.4106,  0.0771,  0.6933,  ..., -0.5881, -0.6007, -0.2466],
        [ 0.2369,  1.1819, -0.0527,  ...,  

In [32]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

print("Documento Permutado:", documento_permutado)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaPermutado = tokenizer.tokenize(sentenca1Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca1TokenizadaPermutado)
embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca1Permutado, tokenizer)
print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaPermutado = tokenizer.tokenize(sentenca2Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca2TokenizadaPermutado)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca2Permutado, tokenizer)
print('\nSentença 2 Permutada=\'', sentenca2Permutado, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaPermutado = tokenizer.tokenize(sentenca3Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca3TokenizadaPermutado)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca3Permutado, tokenizer)
print('\nSentença 3 Permutada=\'', sentenca3Permutado, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaPermutado = tokenizer.tokenize(sentenca4Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca4TokenizadaPermutado)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca4Permutado, tokenizer)
print('\nSentença 4 Permutada=\'', sentenca4Permutado, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Permutado))


Documento Permutado: ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Bom Dia, professor.']

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 1 e término em 7
    Formato modelo : torch.Size([7, 1024])
    Soma embeddings:  -199.66

Sentença 2 Permutada=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 8 e término em 13
    Formato modelo : torch.Size([6, 1024])
    Soma embeddings:  -173.54

Sentença 3 Permutada=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 14 e término em 19
    Formato modelo : torch.Size([6, 1024])
    Soma embeddings:  -167.66

Sentença 4 Permutada=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 20 e término em 24
    Formato modelo : to

### Examinando as sentenças

A mesma sentença apresenta embeddings com valores diferentes, pois se encontram em locais diferentes do documento. A soma de todos os embeddings demonstra isto.

In [33]:
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca4Original[:4]))

print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca1Permutado[:4]))


Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 1024])
    Soma embeddings:  -200.72
    Os 4 primeiros embeddings: tensor([[ 0.4339, -0.7168,  0.1173,  ...,  0.0768, -1.2036,  0.3863],
        [-0.6806, -1.0194, -0.0765,  ..., -0.1768, -0.6326,  0.2895],
        [ 1.1384, -0.8541,  0.8992,  ...,  0.2798, -0.0381,  0.5061],
        [ 0.9206, -0.9862,  0.1347,  ...,  0.4182, -1.0164,  0.4713]])

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 1024])
    Soma embeddings:  -199.66
    Os 4 primeiros embeddings: tensor([[ 0.4472, -0.7700, -0.0164,  ...,  0.3660, -1.3074,  0.4681],
        [-0.6242, -1.0259, -0.2210,  ..., -0.0656, -0.7453,  0.2866],
        [ 1.3008, -1.0530,  0.9163,  ...,  0.3736, -0.1157,  0.4558],
        [ 0.9188, -1.0255,  0.2020

### Diferença entre os embeddings das sentenças

#### Calcula a média da diferença entre os embeddings das sentenças do documento original

In [34]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsub = 0

for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Ssub = torch.sub(mediaEmbeddingSi, mediaEmbeddingSj)

    somaSsub = somaSsub + torch.mean(Ssub)

DsubOriginal = float(somaSsub)/float(len(documento_original)-1)
print("Ssub Original:", DsubOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Ssub Original: 0.0001255537693699201


#### Calcula a média da diferença entre os embeddings das sentenças do documento permutado

In [35]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsub = 0

for i in range(np-1):
     # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Ssub = torch.sub(mediaEmbeddingSi, mediaEmbeddingSj)

    somaSsub = somaSsub + torch.mean(Ssub)

DsubPermutado = float(somaSsub)/float(np-1)
print("Ssub permutado:", DsubPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Bom Dia, professor.']
Quantidade de sentenças: 4
Ssub permutado: -4.3934366355339684e-05


#### Compara as médias da subtração dos embeddings das sentenças do documento original e permutado

In [36]:
print("Dsub Original :", DsubOriginal)
print("Dsub Permutado:", DsubPermutado)

if (DsubOriginal < DsubPermutado):
    print("Documento original tem menor diferença entre as sentenças!")
else:
    print("Documento Permutado tem menor diferença entre as entre as sentenças!")

Dsub Original : 0.0001255537693699201
Dsub Permutado: -4.3934366355339684e-05
Documento Permutado tem menor diferença entre as entre as sentenças!


### Diferença absoluta entre os embeddings das sentenças

#### Calcula a média da diferença absoluta entre os embeddings das sentenças do documento original

In [37]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsubabs = 0

for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Ssubabs = abs(torch.sub(mediaEmbeddingSi, mediaEmbeddingSj))

    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsOriginal = float(somaSsubabs)/float(n-1)
print("Dsubabs Original:", DsubabsOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dsubabs Original: 0.26808643341064453


#### Calcula a média da diferença absoluta entre os embeddings das sentenças do documento permutado

In [38]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsubabs = 0

for i in range(np-1):
     # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Ssubabs = abs(torch.sub(mediaEmbeddingSi, mediaEmbeddingSj))

    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsPermutado = float(somaSsubabs)/float(np-1)
print("Dsubabs permutado:", DsubabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Bom Dia, professor.']
Quantidade de sentenças: 4
Dsubabs permutado: 0.2773038148880005


#### Compara as médias da subtração dos embeddings das sentenças do documento original e permutado

In [39]:
print("Dsubabs Original :", DsubabsOriginal)
print("Dsubabs Permutado:", DsubabsPermutado)

if (DsubabsOriginal < DsubabsPermutado):
    print("Documento original tem menor diferença absoluta entre as sentenças!")
else:
    print("Documento Permutado tem menor diferença absoluta entre as entre as sentenças!")

Dsubabs Original : 0.26808643341064453
Dsubabs Permutado: 0.2773038148880005
Documento original tem menor diferença absoluta entre as sentenças!


### Similaridade de cosseno entre os embeddings das sentenças

In [40]:
# Import das bibliotecas.
from scipy.spatial.distance import cosine

def similaridadeCoseno(sentenca1, sentenca2):
  similaridade = 1 - cosine(sentenca1, sentenca2)
  return similaridade

#### Calcula a média da similaridade de cosseno entre os embeddings das sentenças do documento original

In [41]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsim = 0

for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
    # Similaridade entre os embeddings Si e Sj
    # Entrada: <768 ou 1024>  
    Ssim = similaridadeCoseno(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: <768 ou 1024>
    
    somaSsim = somaSsim + Ssim

DsimOriginal = float(somaSsim)/float(n-1)
print("Dsim Original:", DsimOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dsim Original: 0.7828846176465353


#### Calcula a média da similaridade de cosseno entre as sentenças dos embeddings do documento permutado

In [42]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsim = 0

for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
   # Similaridade entre os embeddings Si e Sj
    # Entrada: <768 ou 1024>  
    Ssim = similaridadeCoseno(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: <768 ou 1024>
    
    somaSsim = somaSsim + Ssim

DsimPermutado = float(somaSsim)/float(np-1)
print("Dsim Original:", DsimPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Bom Dia, professor.']
Quantidade de sentenças: 4
Dsim Original: 0.7676135500272115


#### Compara as médias da similaridade de cosseno dos embeddings das sentenças do documento original e permutado

In [43]:
print("Dsim Original :", DsimOriginal)
print("Dsim Permutado:", DsimPermutado)

if (DsimOriginal > DsimPermutado):
    print("Documento original tem maior similaridade de cosseno entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade de cosseno entre as entre as sentenças!")

Dsim Original : 0.7828846176465353
Dsim Permutado: 0.7676135500272115
Documento original tem maior similaridade de cosseno entre as sentenças!


### Resumo

In [44]:
print("Resultado das medidas utilizando a última camada do BERT")
print("Documento  :   Original            Permutado")
print('Dsub       :   {:.8f}          {:.8f}'.format(DsubOriginal,DsubPermutado))
print('Dubabs     :   {:.8f}          {:.8f}'.format(DsubabsOriginal,DsubabsPermutado))
print('Dsim       :   {:.8f}          {:.8f}'.format(DsimOriginal,DsimPermutado))

Resultado das medidas utilizando a última camada do BERT
Documento  :   Original            Permutado
Dsub       :   0.00012555          -0.00004393
Dubabs     :   0.26808643          0.27730381
Dsim       :   0.78288462          0.76761355


## 7 - Exemplo sentenças de documento original e permutado utilizando embedding a concatenação das 4 últimas camadas do BERT

### Funções auxiliares

In [45]:
def getDocumentoTokenizado(documento, tokenizador):

    # Adiciona os tokens especiais.
    documentoMarcado = "[CLS] " + documento + " [SEP]"

    # Documento tokenizado
    documentoTokenizado = tokenizador.tokenize(documentoMarcado)

    return documentoTokenizado

In [46]:
# Localiza os índices de início e fim de uma sublista em uma lista
def encontrarIndiceSubLista(lista, sublista):
    # Recupera o tamanho da lista 
    h = len(lista)
    # Recupera o tamanho da sublista
    n = len(sublista)    
    skip = {sublista[i]: n - i - 1 for i in range(n - 1)}
    i = n - 1
    while i < h:
        for j in range(n):
            if lista[i - j] != sublista[-j - 1]:
                i += skip.get(lista[i], n)
                break
        else:
            indiceInicio = i - n + 1
            indiceFim = indiceInicio + len(sublista)-1
            return indiceInicio, indiceFim
    return -1, -1

In [47]:
def getEmbeddingSentencaDocumentoEmbedding(embeddingDocumento, documento, sentença, tokenizador):
  # Tokeniza o documento
  documentoTokenizado = getDocumentoTokenizado(documento, tokenizador)  
  #print(documentoTokenizado)

  # Tokeniza a sentença
  sentençaTokenizada = getDocumentoTokenizado(sentença, tokenizador)
  
  # Remove os tokens de início e fim da sentença
  sentençaTokenizada.remove('[CLS]')
  sentençaTokenizada.remove('[SEP]')  
  #print(sentençaTokenizada)
  #print(len(sentençaTokenizada))
  
  # Localiza os índices dos tokens da sentença no documento
  inicio, fim = encontrarIndiceSubLista(documentoTokenizado,sentençaTokenizada)
  #print(inicio,fim) 
 
  # Recupera os embeddings dos tokens da sentença a partir dos embeddings do documento
  embeddingSentença = embeddingDocumento[inicio:fim+1]
  #print("embeddingSentença=", embeddingSentença.shape)
  
  # Retorna o embedding da sentença no documento
  return embeddingSentença

### Documento Original

In [48]:
# Define um sentença de exemplo com diversos significados da palavra  "pilha"
documento_original = ["Bom Dia, professor.",
             "Qual o conteúdo da prova?",              
             "Vai cair tudo na prova?",
             "Aguardo uma resposta, João."]

# Concatena as sentenças do documento
documento_texto_original = ' '.join(documento_original)

# Adiciona os tokens especiais
documento_marcado_original = "[CLS] " + documento_texto_original + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_original = tokenizer.tokenize(documento_marcado_original)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_original = tokenizer.convert_tokens_to_ids(documento_tokenizado_original)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_original, documento_tokens_indexados_original):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Bom           8,399
  2 Dia           3,616
  3 ,               117
  4 professor     2,917
  5 .               119
  6 Qual         13,082
  7 o               146
  8 conteúdo      5,015
  9 da              180
 10 prova         2,310
 11 ?               136
 12 Vai          20,805
 13 cair          9,322
 14 tudo          2,745
 15 na              229
 16 prova         2,310
 17 ?               136
 18 Agu           8,125
 19 ##ardo        2,222
 20 uma             230
 21 resposta      4,299
 22 ,               117
 23 João          1,453
 24 .               119
 25 [SEP]           102


Máscara de atenção das palavras

In [49]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_original = [1] * len(documento_tokenizado_original)

print (mascara_atencao_original)
print (len(mascara_atencao_original))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [50]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_original = torch.as_tensor([documento_tokens_indexados_original])
mascara_atencao_tensores_original = torch.as_tensor([mascara_atencao_original])

Gera os embeddings para o documento original. Guarda todas as camadas da rede em `outputs`.


In [51]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_original, mascara_atencao_tensores_original)

Recupera a saída e concatena as 4 últimas camada do BERT

In [52]:
# Cria uma lista com os tensores a serem concatenados
# Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
# Lista com os tensores a serem concatenados
listaConcat = []
# Percorre os 4 últimos
for i in [-1,-2,-3,-4]:
    # Concatena da lista
    listaConcat.append(outputs[2][i])
    # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
     #print("listaConcat=",len(listaConcat))

# Realiza a concatenação dos embeddings de todos as camadas
# Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
concat4_hidden_states = torch.cat(listaConcat, dim=-1)
# Saída: Entrada: (<1(lote)> x <qtde_tokens> <3072 ou 4096>)  

print ("O vetor da  concatenação das 4 últimas camadas oculta tem o formato:", concat4_hidden_states.size())

O vetor da  concatenação das 4 últimas camadas oculta tem o formato: torch.Size([1, 26, 4096])


Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [53]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
token_embeddings_original = torch.squeeze(concat4_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento original tem o formato:", token_embeddings_original.size())

O vetor de tokens de embedding do documento original tem o formato: torch.Size([26, 4096])


Confirmando vetores dependentes do contexto


In [54]:
for i, token_str in enumerate(documento_tokenizado_original):
  print (i, token_str)

0 [CLS]
1 Bom
2 Dia
3 ,
4 professor
5 .
6 Qual
7 o
8 conteúdo
9 da
10 prova
11 ?
12 Vai
13 cair
14 tudo
15 na
16 prova
17 ?
18 Agu
19 ##ardo
20 uma
21 resposta
22 ,
23 João
24 .
25 [SEP]


Exibe os embenddings das sentenças

In [55]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca1Original, tokenizer)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca2Original, tokenizer)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca3Original, tokenizer)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca4Original, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento original.')

print('\nSentença 1:', sentenca1Original,'-', str(embeddingSentenca1Original[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Original,'-', str(embeddingSentenca2Original[:4]))
print('Soma embedding Sentença2:', sentenca2Original,'-', str(torch.sum(embeddingSentenca2Original[:4])))

print('\nSentença 3:', sentenca3Original,'-', str(embeddingSentenca3Original[:4]))
print('Soma embedding Sentença3:', sentenca3Original,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Original,'-', str(embeddingSentenca4Original[:4]))
print('Soma embedding Sentença4:', sentenca4Original,'-', str(torch.sum(embeddingSentenca4Original[:4])))

Os primeiros 4 valores de cada sentença do documento original.

Sentença 1: Bom Dia, professor. - tensor([[-0.5098, -0.2877,  0.0873,  ...,  0.9288, -0.0796,  0.3684],
        [ 0.6078, -0.8869,  0.3736,  ...,  0.0124,  0.0094, -0.1719],
        [ 0.9075, -1.1233, -0.0093,  ...,  0.1359, -0.3067, -0.4078],
        [ 0.2499, -0.4717, -0.1217,  ...,  0.4604,  0.0075,  0.1555]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-176.3423)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.3987, -0.9450,  0.1785,  ...,  0.8476, -0.0572,  0.0365],
        [ 0.2067, -0.2705,  0.7145,  ...,  0.2113,  0.2899,  0.5135],
        [ 0.2355,  0.2686,  0.5669,  ...,  0.7205,  0.2510,  0.0924],
        [ 0.5264, -0.4600,  0.4810,  ...,  0.1881,  0.0931,  0.1731]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-165.1487)

Sentença 3: Vai cair tudo na prova? - tensor([[ 0.5178,  0.0863,  0.7394,  ..., -0.3301, -0.0115,  0.1168],
        [ 0.1617,  1.1516, -0.0350,  ..., -0.2189,  

Examinando os embeddings do documento original



In [56]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

print("Documento Original:", documento_original)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaOriginal = tokenizer.tokenize(sentenca1Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca1TokenizadaOriginal)
embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca1Original, tokenizer)
print('\nSentença 1 Original=\'', sentenca1Original, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Original))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaOriginal = tokenizer.tokenize(sentenca2Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca2TokenizadaOriginal)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca2Original, tokenizer)
print('\nSentença 2 Original=\'', sentenca2Original, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Original))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaOriginal = tokenizer.tokenize(sentenca3Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca3TokenizadaOriginal)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca3Original, tokenizer)
print('\nSentença 3 Original=\'', sentenca3Original, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Original))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaOriginal = tokenizer.tokenize(sentenca4Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca4TokenizadaOriginal)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, sentenca4Original, tokenizer)
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))


Documento Original: ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']

Sentença 1 Original=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 1 e término em 5
    Formato modelo : torch.Size([5, 4096])
    Soma embeddings:  -225.64

Sentença 2 Original=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 6 e término em 11
    Formato modelo : torch.Size([6, 4096])
    Soma embeddings:  -252.06

Sentença 3 Original=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 12 e término em 17
    Formato modelo : torch.Size([6, 4096])
    Soma embeddings:  -260.65

Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 18 e término em 24
    Formato modelo : torch.S

### Documento Permutado

In [57]:
# Define um sentença de exemplo com diversos significados da palavra  "pilha"
documento_permutado = ["Aguardo uma resposta, João.",
             "Qual o conteúdo da prova?",              
             "Vai cair tudo na prova?",
             "Bom Dia, professor."]

# Use o documento permutado igual ao original para testar se as medidas estão corretas
#documento_permutado = documento_original

# Concatena as sentenças do documento
documento_texto_permutado = ' '.join(documento_permutado)

# Adiciona os tokens especiais
documento_marcado_permutado = "[CLS] " + documento_texto_permutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_permutado = tokenizer.tokenize(documento_marcado_permutado)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_permutado = tokenizer.convert_tokens_to_ids(documento_tokenizado_permutado)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_permutado, documento_tokens_indexados_permutado):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Agu           8,125
  2 ##ardo        2,222
  3 uma             230
  4 resposta      4,299
  5 ,               117
  6 João          1,453
  7 .               119
  8 Qual         13,082
  9 o               146
 10 conteúdo      5,015
 11 da              180
 12 prova         2,310
 13 ?               136
 14 Vai          20,805
 15 cair          9,322
 16 tudo          2,745
 17 na              229
 18 prova         2,310
 19 ?               136
 20 Bom           8,399
 21 Dia           3,616
 22 ,               117
 23 professor     2,917
 24 .               119
 25 [SEP]           102


Máscara de atenção das palavras

In [58]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_permutado = [1] * len(documento_tokenizado_permutado)

print (mascara_atencao_permutado)
print (len(mascara_atencao_permutado))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [59]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_permutado = torch.as_tensor([documento_tokens_indexados_permutado])
mascara_atencao_tensores_permutado = torch.as_tensor([mascara_atencao_permutado])

Gera os embeddings para o documento original. Guarda todas as camadas da rede em `outputs`.


In [60]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_permutado, mascara_atencao_tensores_permutado)

Recupera a saída e concatena as 4 últimas camada do BERT

In [61]:
# Cria uma lista com os tensores a serem concatenados
# Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
# Lista com os tensores a serem concatenados
listaConcat = []
# Percorre os 4 últimos
for i in [-1,-2,-3,-4]:
    # Concatena da lista
    listaConcat.append(outputs[2][i])
    # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
     #print("listaConcat=",len(listaConcat))

# Realiza a concatenação dos embeddings de todos as camadas
# Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
concat4_hidden_states = torch.cat(listaConcat, dim=-1)
# Saída: Entrada: (<1(lote)> x <qtde_tokens> <3072 ou 4096>)  

Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [62]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
token_embeddings_permutado = torch.squeeze(concat4_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento permutado tem o formato:", token_embeddings_permutado.size())

O vetor de tokens de embedding do documento permutado tem o formato: torch.Size([26, 4096])


Exibe os embenddings das sentenças

In [63]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca1Permutado, tokenizer)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca2Permutado, tokenizer)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca3Permutado, tokenizer)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca4Permutado, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento permutado.')

print('\nSentença 1:', sentenca1Permutado,'-', str(embeddingSentenca1Permutado[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Permutado,'-', str(embeddingSentenca2Permutado[:4]))
print('Soma embedding Sentença2:', sentenca2Permutado,'-', str(torch.sum(embeddingSentenca2Permutado[:4])))

print('\nSentença 3:', sentenca3Permutado,'-', str(embeddingSentenca3Permutado[:4]))
print('Soma embedding Sentença3:', sentenca3Permutado,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Permutado,'-', str(embeddingSentenca4Permutado[:4]))
print('Soma embedding Sentença4:', sentenca4Permutado,'-', str(torch.sum(embeddingSentenca4Permutado[:4])))

Os primeiros 4 valores de cada sentença do documento permutado.

Sentença 1: Aguardo uma resposta, João. - tensor([[ 0.4472, -0.7700, -0.0164,  ...,  0.5051, -0.1387,  0.1486],
        [-0.6242, -1.0259, -0.2210,  ...,  0.0995,  0.2928, -0.0829],
        [ 1.3008, -1.0530,  0.9163,  ...,  0.5652,  0.1623,  0.0638],
        [ 0.9188, -1.0255,  0.2020,  ...,  0.5125, -0.5088,  0.1882]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-176.3423)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.3900, -0.9698,  0.1223,  ...,  0.7264, -0.0738,  0.2076],
        [ 0.1376, -0.3465,  0.7442,  ...,  0.1908,  0.3050,  0.5651],
        [ 0.2753,  0.3924,  0.4863,  ...,  0.7431,  0.2877,  0.0924],
        [ 0.5488, -0.5081,  0.4872,  ...,  0.2678,  0.0907,  0.2537]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-164.9317)

Sentença 3: Vai cair tudo na prova? - tensor([[ 0.4106,  0.0771,  0.6933,  ..., -0.2983, -0.0386,  0.1818],
        [ 0.2369,  1.1819, -0.0527,  ..., -

In [64]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

print("Documento Permutado:", documento_permutado)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaPermutado = tokenizer.tokenize(sentenca1Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca1TokenizadaPermutado)
embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca1Permutado, tokenizer)
print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaPermutado = tokenizer.tokenize(sentenca2Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca2TokenizadaPermutado)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca2Permutado, tokenizer)
print('\nSentença 2 Permutada=\'', sentenca2Permutado, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaPermutado = tokenizer.tokenize(sentenca3Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca3TokenizadaPermutado)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca3Permutado, tokenizer)
print('\nSentença 3 Permutada=\'', sentenca3Permutado, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaPermutado = tokenizer.tokenize(sentenca4Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca4TokenizadaPermutado)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, sentenca4Permutado, tokenizer)
print('\nSentença 4 Permutada=\'', sentenca4Permutado, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Permutado))


Documento Permutado: ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Bom Dia, professor.']

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 1 e término em 7
    Formato modelo : torch.Size([7, 4096])
    Soma embeddings:  -292.39

Sentença 2 Permutada=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 8 e término em 13
    Formato modelo : torch.Size([6, 4096])
    Soma embeddings:  -254.00

Sentença 3 Permutada=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 14 e término em 19
    Formato modelo : torch.Size([6, 4096])
    Soma embeddings:  -260.99

Sentença 4 Permutada=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 20 e término em 24
    Formato modelo : to

### Examinando as sentenças

A mesma sentença apresenta embeddings com valores diferentes, pois se encontram em locais diferentes do documento. A soma de todos os embeddings demonstra isto.

In [65]:
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca4Original[:4]))

print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca1Permutado[:4]))


Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 4096])
    Soma embeddings:  -288.69
    Os 4 primeiros embeddings: tensor([[ 0.4339, -0.7168,  0.1173,  ...,  0.3840, -0.0936,  0.1355],
        [-0.6806, -1.0194, -0.0765,  ...,  0.2201,  0.2731, -0.0384],
        [ 1.1384, -0.8541,  0.8992,  ...,  0.5247,  0.2278,  0.0653],
        [ 0.9206, -0.9862,  0.1347,  ...,  0.4393, -0.4792,  0.1743]])

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 4096])
    Soma embeddings:  -292.39
    Os 4 primeiros embeddings: tensor([[ 0.4472, -0.7700, -0.0164,  ...,  0.5051, -0.1387,  0.1486],
        [-0.6242, -1.0259, -0.2210,  ...,  0.0995,  0.2928, -0.0829],
        [ 1.3008, -1.0530,  0.9163,  ...,  0.5652,  0.1623,  0.0638],
        [ 0.9188, -1.0255,  0.2020

### Diferença entre os embeddings das sentenças

#### Calcula a média da diferença entre os embeddings das sentenças do documento original

In [66]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsub = 0

for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Ssub = torch.sub(mediaEmbeddingSi, mediaEmbeddingSj)

    somaSsub = somaSsub + torch.mean(Ssub)

DsubOriginal = float(somaSsub)/float(n-1)
print("Dsub Original:", DsubOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dsub Original: -0.0003163199095676343


#### Calcula a média da diferença entre os embeddings das sentenças do documento permutado

In [67]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsub = 0

for i in range(np-1):
     # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Ssub = torch.sub(mediaEmbeddingSi, mediaEmbeddingSj)

    somaSsub = somaSsub + torch.mean(Ssub)

DsubPermutado = float(somaSsub)/float(np-1)
print("Dsub permutado:", DsubPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Bom Dia, professor.']
Quantidade de sentenças: 4
Dsub permutado: 0.00024482708734770614


#### Compara as médias da subtração dos embeddings das sentenças do documento original e permutado

In [68]:
print("Dsub Original :", DsubOriginal)
print("Dsub Permutado:", DsubPermutado)

if (DsubOriginal < DsubPermutado):
    print("Documento original tem menor diferença entre as sentenças!")
else:
    print("Documento Permutado tem menor diferença entre as entre as sentenças!")

Dsub Original : -0.0003163199095676343
Dsub Permutado: 0.00024482708734770614
Documento original tem menor diferença entre as sentenças!


### Diferença absoluta entre os embeddings das sentenças

#### Calcula a média da diferença absoluta entre os embeddings das sentenças do documento original

In [69]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsubabs = 0

for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Ssubabs = abs(torch.sub(mediaEmbeddingSi, mediaEmbeddingSj))

    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsOriginal = float(somaSsubabs)/float(n-1)
print("Dsubabs Original:", DsubabsOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dsubabs Original: 0.22927141189575195


#### Calcula a média da diferença absoluta entre os embeddings das sentenças do documento permutado

In [70]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsubabs = 0

for i in range(np-1):
     # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Ssubabs = abs(torch.sub(mediaEmbeddingSi, mediaEmbeddingSj))

    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsPermutado = float(somaSsubabs)/float(np-1)
print("Ssubabs permutado:", DsubabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Bom Dia, professor.']
Quantidade de sentenças: 4
Ssubabs permutado: 0.23909521102905273


#### Compara as médias da subtração dos embeddings das sentenças do documento original e permutado

In [71]:
print("Dsubabs Original :", DsubabsOriginal)
print("Dsubabs Permutado:", DsubabsPermutado)

if (DsubabsOriginal < DsubabsPermutado):
    print("Documento original tem menor diferença absoluta entre as sentenças!")
else:
    print("Documento Permutado tem menor diferença absoluta entre as entre as sentenças!")

Dsubabs Original : 0.22927141189575195
Dsubabs Permutado: 0.23909521102905273
Documento original tem menor diferença absoluta entre as sentenças!


### Similaridade de cosseno entre os embeddings das sentenças

In [72]:
# Import das bibliotecas.
from scipy.spatial.distance import cosine

def similaridadeCoseno(sentenca1, sentenca2):
  similaridade = 1 - cosine(sentenca1, sentenca2)
  return similaridade

#### Calcula a média da similaridade de cosseno entre os embeddings das sentenças do documento original

In [73]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsim = 0

for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_original, documento_texto_original, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
    # Similaridade entre os embeddings Si e Sj
    # Entrada: <768 ou 1024>  
    Ssim = similaridadeCoseno(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: <768 ou 1024>
    
    somaSsim = somaSsim + Ssim

DsimOriginal = float(somaSsim)/float(n-1)
print("Dsim Original:", DsimOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dsim Original: 0.8178287347157797


#### Calcula a média da similaridade de cosseno entre as sentenças dos embeddings do documento permutado

In [74]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsim = 0

for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Si, tokenizer)
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, Sj, tokenizer)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSi=", mediaCamadasSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaCamadasSj=", mediaCamadasSj.shape)
  
   # Similaridade entre os embeddings Si e Sj
    # Entrada: <768 ou 1024>  
    Ssim = similaridadeCoseno(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: <768 ou 1024>
    
    somaSsim = somaSsim + Ssim

DsimPermutado = float(somaSsim)/float(np-1)
print("Dsim Original:", DsimPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Bom Dia, professor.']
Quantidade de sentenças: 4
Dsim Original: 0.8008809288342794


#### Compara as médias da similaridade de cosseno dos embeddings das sentenças do documento original e permutado

In [75]:
print("Dsim Original :", DsimOriginal)
print("Dsim Permutado:", DsimPermutado)

if (DsimOriginal > DsimPermutado):
    print("Documento original tem maior similaridade de cosseno entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade de cosseno entre as entre as sentenças!")

Dsim Original : 0.8178287347157797
Dsim Permutado: 0.8008809288342794
Documento original tem maior similaridade de cosseno entre as sentenças!


### Resumo

- Resultado das medidas utilizando a última camada do BERT
- Documento  :   Original            Permutado
- Ssub       :   0.00012555          -0.00004393
- Subabs     :   0.26808643          0.27730381
- Ssim       :   0.78288462          0.76761355

In [76]:
print("Resultado das medidas utilizando a concatenação das 4 últimas camadas do BERT")
print("Documento  :   Original            Permutado")
print('Dsub       :   {:.8f}          {:.8f}'.format(DsubOriginal,DsubPermutado))
print('Dubabs     :   {:.8f}          {:.8f}'.format(DsubabsOriginal,DsubabsPermutado))
print('Dsim       :   {:.8f}          {:.8f}'.format(DsimOriginal,DsimPermutado))


Resultado das medidas utilizando a concatenação das 4 últimas camadas do BERT
Documento  :   Original            Permutado
Dsub       :   -0.00031632          0.00024483
Dubabs     :   0.22927141          0.23909521
Dsim       :   0.81782873          0.80088093
